# Importing packages

In [2]:
# Install required packages for Albert model
!pip install -q sentencepiece
!pip install -q transformers
!pip install -q tokenizers
!pip install -qU hazm
!pip install -qU clean-text[gpl]


#!pip install git+https://github.com/LIAAD/yake

!pip install rank_bm25

!pip install -qU sentence-transformers
!pip install -qU wikipedia-api


!pip install rank_bm25


!mkdir resources
!wget -q "https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip" -P resources
!unzip -qq resources/resources-0.5.zip -d resources

!pip install faiss-cpu

!rm -rf /content/4ccae468eb73bf6c4f4de3075ddb5336
!rm -rf /content/preproc
!rm preprocessing.py utils.py
!mkdir -p /content/preproc
!git clone https://gist.github.com/4ccae468eb73bf6c4f4de3075ddb5336.git /content/preproc/
!mv /content/preproc/* /content/
!rm -rf /content/preproc




     |████████████████████████████████| 1.2MB 5.8MB/s 
     |████████████████████████████████| 1.8MB 5.8MB/s 
     |████████████████████████████████| 3.2MB 45.6MB/s 
     |████████████████████████████████| 890kB 40.9MB/s 
     |████████████████████████████████| 317kB 4.3MB/s 
     |████████████████████████████████| 1.4MB 21.6MB/s 
     |████████████████████████████████| 235kB 33.0MB/s 
     |████████████████████████████████| 71kB 3.5MB/s 
     |████████████████████████████████| 133kB 7.5MB/s 
     |████████████████████████████████| 245kB 24.7MB/s 
     |████████████████████████████████| 71kB 3.6MB/s 
     |████████████████████████████████| 8.2MB 4.4MB/s 
rm: cannot remove 'preprocessing.py': No such file or directory
rm: cannot remove 'utils.py': No such file or directory
Cloning into '/content/preproc'...
remote: Enumerating objects: 7, done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 7
Unpacking objects: 100% (7/7), done.


In [3]:
import numpy as np 
import pandas as pd
import re
from tqdm import tqdm 
import os
# import yake
from hazm import stopwords_list
from __future__ import unicode_literals
from hazm import *
import pickle
import requests
from termcolor import colored
from preprocessing import cleaning

import time

import plotly.express as px
import plotly.graph_objects as go
from itertools import chain
# for the models
import tensorflow as tf
import matplotlib.pyplot as plt
import re
import string


# BERT base
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from __future__ import unicode_literals
import torch.nn.functional as FloatingPointError

Base_BERT_Path = 'HooshvareLab/bert-fa-base-uncased'
import faiss

# evaluator
from transformers import BertConfig, BertTokenizer
from transformers import TFBertModel, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features
from sklearn.model_selection import StratifiedKFold


# sentence BERT
from sentence_transformers import models, SentenceTransformer, util

In [4]:
bert_tokenizer = BertTokenizer.from_pretrained(Base_BERT_Path)


# 📲 Loading the dataset


In [20]:
from google.colab import drive
drive.mount('/content/drive')


data_address = '/content/drive/MyDrive/COVID-PSS.xls'
keys_address = '/content/drive/MyDrive/keywords_final_distilled_NE (1).pickle'
cleaned_titles_address = '/content/drive/MyDrive/title_cleaned_without_corona_2.pkl'

df = pd.read_csv(data_address)
list_t = pd.read_pickle(cleaned_titles_address)

keywords = pd.read_pickle(keys_address)
keywords = [v for k,v in keywords.items()]



assert len(keywords) == len(df)
df['keywords'] = keywords
df.drop(columns=['img', 'link'], inplace=True)


tfidf_results = pd.read_pickle('/content/drive/MyDrive/CoPer paper-Models/Results/TFIDF_Ranked.pkl')
sbert_results = pd.read_pickle('/content/drive/MyDrive/CoPer paper-Models/Results/sbert-WikiNli-DifferentWeights_FineTuned_f300-Ranked_2.pkl')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Helpers

In [14]:

moses = ['است', 'بود', 'شد', 'گشت', 'گردید']
correct_POS = {'AJ': 'ADJ', 'PRO': 'PR', 'P': 'PREP', 'Ne': 'N', 'AJe': 'ADJ'}


def cal_wordiness_level(question, SBJ_answer, OBJ_answer, PP_answer, MOS_answer, V_answer):
  wordiness_level = 0
  if SBJ_answer != None:
    wordiness_level += 20
  if bert_tokenizer.tokenize(question)[-1] in moses:
    if SBJ_answer != None:
      wordiness_level += 20
  if PP_answer != None:
    wordiness_level += 10
  if V_answer != None:
    wordiness_level += 20
  return wordiness_level


def cal_wordiness_ratio(wordiness_level):
  wordiness_ratio = None
  if wordiness_level <= 20:
    wordiness_ratio = 1
  elif 20 < wordiness_level < 30:
    wordiness_ratio = 0.80
  elif wordiness_level == 30:
    wordiness_ratio = 0.60
  elif 30 < wordiness_level < 40:
    wordiness_ratio = 0.35
  elif wordiness_level >= 40:
    wordiness_ratio = 0
  return wordiness_ratio


correct_POS = {'AJ': 'ADJ', 'PRO': 'PR', 'P': 'PREP', 'Ne': 'N', 'AJe': 'ADJ'}

SBJ_pattern = '^(?!PREP)(PREM|PRENUM(PREP(ADJ)?)?)?((N((ADJ)|(N)+)*(ADJ|N|PR))|(N(ADJ)?)|(PR))(?!POSTP)(CONJ)?'
OBJ_pattern = '^(?!PREP)(PREM|PRENUM(PREP(ADJ)?)?)?((N((ADJ)|(N)+)*(ADJ|N|PR))|(N(ADJ)?)|(PR))(POSTP)'
PP_pattern = '(PREP)(PREM|PRENUM(PREP(ADJ)?)?)?((N((ADJ)|(N)+)*(ADJ|N|PR))|(N(ADJ)?)|(PR))(?!POSTP)(CONJ)?'
MOS_pattern = '^(?!PREP)(PREM|PRENUM(PREP(ADJ)?)?)?((N((ADJ)|(N)+)*(ADJ|N|PR))|(N(ADJ)?)|(PR))(?!POSTP)(CONJ)?'
V_pattern = '(V)*V'

tagger = POSTagger(model='resources/postagger.model')


In [10]:
def tfidf_ratio(question):
    tokenized_question = bert_tokenizer.tokenize(question)
    sentence_tagged = tagger.tag(tokenized_question)
    tag_query = ''.join([each[1] if each[1] not in correct_POS else correct_POS[each[1]] for each in sentence_tagged])

    SBJ_answer = re.search(SBJ_pattern, tag_query)
    OBJ_answer = re.search(OBJ_pattern, tag_query)
    PP_answer = re.search(PP_pattern, tag_query)
    MOS_answer = re.search(MOS_pattern, tag_query)
    V_answer = re.search(V_pattern, tag_query)

    wordiness_level = cal_wordiness_level(question, SBJ_answer, OBJ_answer, PP_answer, MOS_answer, V_answer)
    wordiness_ratio = cal_wordiness_ratio(wordiness_level)
    return wordiness_ratio
    

In [29]:
def get_results(tfidf_results, sbert_results, top_n = 50):

    """Takes in both sides-tfidf and sbert then outputs a score"""

    # check if both have same number of results
    assert len(tfidf_results) == len(sbert_results)

    results = []
    for i in range(len(tfidf_results)):
        print(f'question {i+1} ')
        # checking the question
        assert tfidf_results[i]['question'] == sbert_results[i]['question']
        assert len(tfidf_results[i]['index']) == len(sbert_results[i]['index'])

        question = tfidf_results[i]['question']

        wordiness = tfidf_ratio(question)

        # creating a dataframe for each question

        # getting the indices for each bm_selected record
        #  in order for each model
        tfidf_indices = tfidf_results[i]['index']
        sbert_indices = sbert_results[i]['index']


        # getting the scores for each record
        tfidf_scores = tfidf_results[i]['score']
        sbert_scores = sbert_results[i]['score']

        # standardizing scores

        # tfidf
        tfidf_max = max(tfidf_scores)
        tfidf_scores = tfidf_scores/tfidf_max

        # sbert
        bert_max = max(sbert_scores)
        sbert_scores = sbert_scores/bert_max


        # making a df for each
        assert len(tfidf_indices) == len(tfidf_scores)


        df_scores_tfidf = pd.DataFrame(np.c_[tfidf_indices, tfidf_scores], columns = ['indices', 'scores_tfidf'])
        df_scores_sbert = pd.DataFrame(np.c_[sbert_indices, sbert_scores], columns = ['indices', 'scores_sbert'])


        # merging them based on the indices
        df_all = pd.merge(df_scores_tfidf, df_scores_sbert,
          on = 'indices').sort_values(by='scores_tfidf')


        # creating an overall score
        df_all.loc[:, 'overall_score'] = 0.9 * df_all.scores_sbert + 0.1 * ((wordiness * df_all.scores_tfidf) + ((1 - wordiness) * (df_all.scores_sbert)))

        # sorting them and getting the top results
        df_all = df_all.sort_values(by='overall_score', ascending=False)
        
        top_k = df_all.indices.values[:top_n]

        results.append({'question': question,
                        'wordiness_rate': wordiness,
                        'index': top_k})
        
    return results, df_all[:top_n]

# Get the results

In [30]:
results, df_all = get_results(tfidf_results, sbert_results, top_n = 50)

question 1 
question 2 
question 3 
question 4 
question 5 
question 6 
question 7 
question 8 
question 9 
question 10 
question 11 
question 12 
question 13 
question 14 
question 15 
question 16 
question 17 
question 18 
question 19 
question 20 
question 21 
question 22 
question 23 
question 24 
question 25 
question 26 
question 27 
question 28 
question 29 
question 30 
question 31 
question 32 
question 33 
question 34 
question 35 
question 36 
question 37 
question 38 
question 39 
question 40 
question 41 
question 42 
question 43 
question 44 
question 45 
question 46 
question 47 
question 48 
question 49 
question 50 


In [31]:
with open('/content/drive/MyDrive/CoPer paper-Models/Results/Ranked_BM25_TFIDF_sbert-WikiNli-DifferentWeights_FineTuned_f300_Ranked_2-0.9s.pkl', 'wb') as f:
    pickle.dump(results, f)